<a href="https://colab.research.google.com/github/eisbetterthanpi/latent-flow-model/blob/main/U_DiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title RoPE RotEmb
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RoPE(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, seq_len=512, base=10000):
        super().__init__()
        self.dim, self.base = dim, base
        theta = 1.0 / (base ** (torch.arange(0, dim, step=2) / dim))
        pos = torch.arange(seq_len).unsqueeze(-1)
        angles = (pos * theta)[None,...,None] # [seq_len, 1] * [dim // 2] -> [1, seq_len, dim // 2, 1]
        self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [1, seq_len, dim // 2, 2] -> [1, seq_len, dim]

    def forward(self, x): # [batch, T, dim] / [batch, T, n_heads, d_head]
        seq_len = x.size(1)
        if self.rot_emb.shape[0] < seq_len: self.__init__(self.dim, seq_len, self.base)
        # print('rope fwd', x.shape, self.rot_emb.shape)
        if x.dim()==4: return x * self.rot_emb[:,:seq_len].unsqueeze(2)
        return x * self.rot_emb[:,:seq_len]

class RotEmb(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, top=torch.pi, base=10000):
        super().__init__()
        self.theta = top / (base ** (torch.arange(0, dim, step=2, device=device) / dim))
        # self.theta = top / (base ** torch.linspace(0, 1, dim//2, device=device))

    def forward(self, pos): # [batch] in [0,1]
        angles = (pos.unsqueeze(-1) * self.theta).unsqueeze(-1) # [seq_len, 1] * [dim // 2] -> [seq_len, dim // 2, 1]
        rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1) # [seq_len, dim // 2, 2]
        return rot_emb.flatten(-2) # [seq_len, dim]


In [13]:
# @title YuchuanTian/U-DiT down
# https://github.com/YuchuanTian/U-DiT/blob/main/udit_models.py
# https://arxiv.org/pdf/2405.02730v1
import torch
import torch.nn as nn
import torch.nn.functional as F

def zero_module(module):
    """Zero out the parameters of a module and return it."""
    for p in module.parameters():
        p.detach().zero_()
    return module
import inspect

class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
        for layer in self:
            params = inspect.signature(layer.forward).parameters.keys()
            layer._fwdparams = ','.join(params)

    def forward(self, x, cond=None):
        for layer in self:
            args = [x]
            if 'cond' in layer._fwdparams: args.append(cond)
            x = layer(*args)
        return x

# # class LayerNorm2d(nn.LayerNorm):
# class LayerNorm2d(nn.RMSNorm):
#     def __init__(self, num_channels, eps=1e-6, affine=True):
#         super().__init__(num_channels, eps=eps, elementwise_affine=affine)

#     def forward(self, x):
#         x = x.permute(0, 2, 3, 1)
#         # x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
#         x = F.rms_norm(x, self.normalized_shape, self.weight, self.eps)
#         # rms_norm(input, normalized_shape, weight=None, eps=None)
#         x = x.permute(0, 3, 1, 2)
#         return x


class RMSNorm2d(nn.RMSNorm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x = super().forward(x)
        x = x.permute(0, 3, 1, 2)
        return x

from einops import rearrange

class DownSampler(nn.Module):
    def __init__(self, dim, kernel_size=5, r=2):
        super().__init__()
        self.r = r
        self.layer = nn.Conv2d(dim, dim, kernel_size, 1, kernel_size//2, groups=dim)
    def forward(self, x):
        b,c,h,w = x.shape
        x = self.layer(x) + x # down_shortcut
        # return rearrange(x, 'b d (h dh) (w dw) -> b (dh dw) (h w) d', dh=self.r, dw=self.r)
        return x.reshape(b,c,h//self.r, self.r, w//self.r, self.r).permute(0,3,5,2,4,1).flatten(1,2).flatten(2,3) # [b,c,h*dh,w*dw] -> [b,dh*dw,h*w,c]

# conv res pixeldown

class DownSample_Attn(nn.Module):
    def __init__(self, dim, down_factor, num_heads):
        super().__init__()
        # self.input_resolution = (input_size, input_size)
        self.dim = dim
        self.heads = num_heads
        head_dim = dim // num_heads
        self.dh, self.dw = down_factor, down_factor
        self.to_qkv = nn.Linear(dim, dim*3, bias=False)
        self.to_out = nn.Conv2d(dim, dim, 1)
        self.rope = RoPE(head_dim, seq_len=512, base=10000)
        # self.scale = head_dim**-.5 # v1
        # v2
        self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)
        # self.logit_scale = nn.Parameter(10 * torch.ones((num_heads, 1, 1)))

        self.downsampler = DownSampler(dim, r=down_factor)

    def forward(self, x): # [b,c,h,w]
        b, _, h, w = x.size()
        x = self.downsampler(x) # [b, r^2, h/r*w/r, c] = [b, r^2, N, c]
        q,k,v = self.to_qkv(x).chunk(3, dim=-1)

        N = h * w // self.dh // self.dw
        # self.freqs_cis = precompute_freqs_cis_2d(self.dim // self.heads, N).to(x.device)
        # q, k input shape: B N H Hc
        q, k = map(lambda t: rearrange(t, 'b p n (h d) -> (b p) n h d', h=self.heads), [q,k]) # [b, r^2, h/r*w/r, d] -> [b*r^2, h/r*w/r, n_heads, d_head]
        v = rearrange(v, 'b p n (h d) -> b p h n d', h=self.heads)
        # q, k = apply_rotary_emb(q, k, freqs_cis=self.freqs_cis)
        q, k = self.rope(q), self.rope(k)
        q, k = rearrange(q, '(b p) n h d -> b p h n d', b=b), rearrange(k, '(b p) n h d -> b p h n d', b=b)

        # # v1 attention
        # attn = (q @ k.transpose(-2, -1))
        # attn = attn * self.scale
        # v2 attention
        attn = (F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(-2, -1))
        attn = attn * torch.clamp(self.logit_scale, max=4.6052).exp()
        # attn = attn * torch.clamp(self.logit_scale, max=100)

        attn = attn.softmax(dim=-1)
        x = (attn @ v)
        # upsample
        x = rearrange(x, 'b (dh dw) he (h w) d -> b (he d) (h dh) (w dw)', h=h // self.dh, w=w // self.dw, dh=self.dh, dw=self.dw)
        x = self.to_out(x)
        return x

class FeedForward(nn.Module):
    def __init__(self, dim, ff_mult=1):
        super().__init__()
        d_model = dim*ff_mult
        self.project_in = nn.Sequential(nn.Conv2d(dim, d_model, kernel_size=1), nn.GELU())
        self.dwconv = nn.ModuleList([
            # nn.Conv2d(d_model, d_model, 5, 1, 5//2, groups=d_model),
            nn.Conv2d(d_model, d_model, 3, 1, 3//2, groups=d_model),
            nn.Conv2d(d_model, d_model, 1, 1, 1//2, groups=d_model),
        ])
        self.project_out = nn.Sequential(nn.Conv2d(d_model, d_model//2, kernel_size=1),
            nn.Conv2d(d_model//2, dim, kernel_size=1))

    def forward(self, x):
        x = self.project_in(x) # no
        x = x + sum([conv(x) for conv in self.dwconv])
        x = self.project_out(x)
        return x


# class SEBlock(nn.Module): # https://github.com/DingXiaoH/RepVGG/blob/main/se_block.py#L7
#     def __init__(self, in_ch, dim=None):
#         super().__init__()
#         dim = dim or in_ch//4
#         self.se = nn.Sequential( # [b,c,h,w] -> [b,c,1,1]
#             nn.AdaptiveAvgPool2d((1,1)),
#             nn.Conv2d(in_ch, dim, 1, 1), nn.ReLU(),
#             nn.Conv2d(dim, in_ch, 1, 1), nn.Sigmoid(),
#         )
#     def forward(self, x): # [b,c,h,w]
#         return x * self.se(x) # [b,c,h,w]

# # https://github.com/DingXiaoH/RepVGG/blob/main/repvgg.py
# # https://github.com/DingXiaoH/RepVGG/blob/main/repvggplus.py#L28
# class FeedForward(nn.Module):
#     def __init__(self, in_ch, out_ch=None, ff_mult=1):
#         super().__init__()
#         # d_model = dim*ff_mult
#         out_ch = out_ch or in_ch
#         self.dwconv = nn.ModuleList([
#             nn.Sequential(nn.Conv2d(in_ch, out_ch, 3, 1, 3//2), nn.BatchNorm2d(out_ch)),
#             nn.Sequential(nn.Conv2d(in_ch, out_ch, 1, 1, 1//2), nn.BatchNorm2d(out_ch)),
#         ])
#         self.project_out = nn.Sequential(nn.GELU(), SEBlock(out_ch, out_ch//4)) # act, SEblock

#     def forward(self, x):
#         x = x + sum([conv(x) for conv in self.dwconv])
#         x = self.project_out(x)
#         return x

# # me
# class FeedForward(nn.Module):
#     def __init__(self, d_model, ff_mult=1):
#         super().__init__()
#         # d_model = dim*ff_mult
#         self.project_in = nn.Sequential(nn.BatchNorm2d(d_model), nn.GELU())
#         self.dwconv = nn.ModuleList([
#             nn.Conv2d(d_model, d_model, 3, 1, 3//2),
#             nn.Conv2d(d_model, d_model, 1, 1, 1//2),
#         ])
#         # self.project_out = nn.Sequential(nn.GELU(), nn.Conv2d(d_model, dim, kernel_size=1)) # act, SEblock

#     def forward(self, x):
#         h = self.project_in(x) # no
#         x = x + sum([conv(h) for conv in self.dwconv])
#         # x = self.project_out(out)
#         return x

class TimestepEmbedder(nn.Module):
    def __init__(self, hidden_size, emb_dim=256):
        super().__init__()
        self.rot_emb = RotEmb(emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim, hidden_size), nn.SiLU(),
            nn.Linear(hidden_size, hidden_size),
        )
        # nn.init.normal_(self.mlp.weight, std=0.02)
        self.mlp.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, std=0.02)
    def forward(self, t):
        t_freq = self.rot_emb(t)
        t_emb = self.mlp(t_freq)
        return t_emb



class U_DiTBlock(nn.Module):
    """A IPT block with adaptive layer norm zero (adaLN-Zero) conIPTioning."""
    def __init__(self, hidden_size, num_heads, mlp_ratio=2, down_factor=2):
        super().__init__()
        self.norm1 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        self.attn = DownSample_Attn(hidden_size, down_factor=down_factor, num_heads=num_heads)
        self.norm2 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        # self.mlp = FeedForward(hidden_size, mlp_ratio)
        self.mlp = FeedForward(hidden_size)
        # print('U_DiTBlock init', hidden_size)
        self.hidden_size = hidden_size
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(), zero_module(nn.Linear(hidden_size, 6 * hidden_size))
        )

    def forward(self, x, cond):
        # print('U_DiTBlock', x.shape, self.hidden_size, cond.shape)
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation(cond).chunk(6, dim=1)
        x = x + gate_msa[...,None,None] * self.attn(modulate(self.norm1(x), shift_msa, scale_msa))
        x = x + gate_mlp[...,None,None] * self.mlp(modulate(self.norm2(x), shift_mlp, scale_mlp))
        return x

def modulate(x, shift, scale):
    return x * (1 + scale[...,None,None]) + shift[...,None,None]


class FinalLayer(nn.Module):
    def __init__(self, hidden_size, out_channels):
        super().__init__()
        self.in_proj = nn.Conv2d(hidden_size, hidden_size, kernel_size=3, stride=1, padding=1)
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(), zero_module(nn.Linear(hidden_size, 2*hidden_size))
        )
        self.norm_final = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        self.out_proj = zero_module(nn.Conv2d(hidden_size, out_channels, kernel_size=3, stride=1, padding=1))

    def forward(self, x, c):
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=1)
        x = self.in_proj(x)
        x = modulate(self.norm_final(x), shift, scale)
        x = self.out_proj(x)
        return x



class UpDownsample(nn.Module):
    def __init__(self, n_feat, r=1):
        super().__init__()
        if r>1: sample = nn.PixelShuffle(r)
        elif r<1: sample = nn.PixelUnshuffle(int(1/r))
        else: sample = nn.Identity()
        self.body = nn.Sequential(nn.Conv2d(n_feat, int(n_feat*r), 3, 1, 1, bias=False), sample)
    def forward(self, x): return self.body(x)

class levelBlock(nn.Module):
    def __init__(self, hidden_size, num_heads, depth=1, updown=False, red=False):
        super().__init__()

        if updown=='down': level = [
            UpDownsample(hidden_size, r=1/2),
            *[U_DiTBlock(hidden_size*2, num_heads) for _ in range(depth)]
        ]
        elif updown=='up': level = [
            # nn.Conv2d(hidden_size, hidden_size//2, kernel_size=1),
            # *[U_DiTBlock(hidden_size//2, num_heads) for _ in range(depth)],
            # UpDownsample(hidden_size//2, r=2)
            nn.Conv2d(hidden_size*2, hidden_size, kernel_size=1),
            *[U_DiTBlock(hidden_size, num_heads) for _ in range(depth)],
            UpDownsample(hidden_size, r=2)
            ]
        else: level = [
            nn.Conv2d(hidden_size*2, hidden_size, kernel_size=1) if red else nn.Identity(),
            *[U_DiTBlock(hidden_size, num_heads) for _ in range(depth)]
        ]
        self.seq = Seq(*level)

    def forward(self, x, cond):
        return self.seq(x, cond)


class U_DiT(nn.Module):
    """Diffusion UNet model with a Transformer backbone."""
    def __init__(self, in_channels=3, hidden_size=1152, depth=[2,5,8,5,2], num_heads=16, cond_dim=16):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = in_channels #* 2 if learn_sigma else in_channels
        self.num_heads = num_heads

        self.x_embedder = nn.Conv2d(in_channels, hidden_size, kernel_size=3, stride=1, padding=1)
        # Initialize patch_embed like nn.Linear (instead of nn.Conv2d):
        w = self.x_embedder.weight.data
        nn.init.xavier_uniform_(w.view([w.shape[0], -1]))
        nn.init.constant_(self.x_embedder.bias, 0)

        self.hidden_size = hidden_size
        self.t_embedder = TimestepEmbedder(hidden_size*(1+2+4))
        self.y_embedder = nn.Linear(cond_dim, hidden_size*(1+2+4))

        self.down_list = nn.ModuleList([
            levelBlock(hidden_size, num_heads, depth=1, updown=False),
            levelBlock(hidden_size, num_heads, depth=1, updown='down'),
        ])
        self.middle_block = Seq(
            UpDownsample(hidden_size*2, r=1/2),
            # levelBlock(hidden_size*4, num_heads, depth=1, updown=False),
            *[U_DiTBlock(hidden_size*4, num_heads) for _ in range(depth[2])],
            UpDownsample(hidden_size*4, r=2),
        )
        self.up_list = nn.ModuleList([
            levelBlock(hidden_size*2, num_heads, depth=1, updown='up'),
            levelBlock(hidden_size*1, num_heads, depth=1, updown=False, red=True),
        ])

        self.final_layer = FinalLayer(hidden_size, self.out_channels)
        self.apply(self._basic_init)

    def _basic_init(self, module):
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)


    def forward(self, x, t, y): # [b,c,h,w], time [b], class label [b]
        x = self.x_embedder(x) # (N, C, H, W)
        c123 = self.t_embedder(t) + self.y_embedder(y)
        cond = c123.split([self.hidden_size, self.hidden_size*2, self.hidden_size*4], dim=1)

        blocks = []
        for i, down in enumerate(self.down_list):
            x = down(x, cond[i])
            blocks.append(x)
        x = self.middle_block(x, cond[-1])
        for i, up in enumerate(self.up_list):
            x = torch.cat([x, blocks[-i-1]*2**.5], dim=1) # scale residuals by 1/sqrt2
            x = up(x, cond[-2-i]) # x = up(x, blocks[-i - 1])

        x = self.final_layer(x, cond[0]) # (N, T, patch_size ** 2 * out_channels)
        return x



# def U_DiT_S(**kwargs):
#     return U_DiT(down_factor=2, hidden_size=96, num_heads=4, depth=[2,5,8,5,2], mlp_ratio=2, downsampler='dwconv5', down_shortcut=1)
# def U_DiT_B(**kwargs): return U_DiT(down_factor=2, hidden_size=192, num_heads=8,
# def U_DiT_L(**kwargs): return U_DiT(down_factor=2, hidden_size=384, num_heads=16,

cond_dim=16
# model = U_DiT_S()
model = U_DiT(in_channels=3, hidden_size=16, num_heads=4, depth=[2,3,3,3,2], cond_dim=cond_dim).to(device)

batch=4
inputs = torch.rand(batch, 3, 32, 32)
t = torch.rand(batch)
y = torch.rand(batch, cond_dim)

model(inputs, t, y)
out = model(inputs, t, y)
print(out.shape)

optim = torch.optim.AdamW(model.parameters(), lr=1e-3) #
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683


# model.train()
# out = model(inputs, t, y)
# gt = torch.rand(1, 8, 32, 32)
# loss = torch.mean(out-gt)
# loss.backward()


torch.Size([4, 3, 32, 32])
339503


In [11]:

class RMSNorm2d(nn.RMSNorm):
    # def __init__(self, num_channels, **kwargs):
    #     super().__init__(num_channels, **kwargs)
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x = super().forward(x)
        x = x.permute(0, 3, 1, 2)
        return x

x = torch.rand(3, 2,3,3)
norm=RMSNorm2d(2)
y=norm(x)
# print(y)

x = x.permute(0, 2, 3, 1)
# x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
x = F.rms_norm(x, (2,))
# rms_norm(input, normalized_shape, weight=None, eps=None)
x = x.permute(0, 3, 1, 2)

print((x==y).all())

tensor(True)


In [ ]:
 import torch
x = torch.rand(3, 2,8,8)
b, c, h, w = x.shape
r=2
from einops import rearrange
y = rearrange(x, 'b d (h dh) (w dw) -> b (dh dw) (h w) d', dh=r, dw=r)
z = x.reshape(b,c,h//r, r, w//r, r).permute(0,3,5,2,4,1).flatten(1,2).flatten(2,3) # [b,c,h*dh,w*dw] -> [b,dh*dw,h*w,c]
p=functional.pixel_unshuffle(x, 2).unflatten(1,(c,r**2)).flatten(3).permute(0,2,)

print(y)
print(z)


In [ ]:
print((y==z).all())

tensor(True)


In [ ]:

# @title YuchuanTian/U-DiT
# https://github.com/YuchuanTian/U-DiT/blob/main/udit_models.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class LayerNorm2d(nn.LayerNorm):
    def __init__(self, num_channels, eps=1e-6, affine=True):
        super().__init__(num_channels, eps=eps, elementwise_affine=affine)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        x = x.permute(0, 3, 1, 2)
        return x


#######################################3###################################
def precompute_freqs_cis_2d(dim, end, theta = 10000.0, scale=1.0, use_cls=False):
    H = int( end**0.5 )
    # assert  H * H == end
    flat_patch_pos = torch.arange(0 if not use_cls else -1, end) # N = end
    x_pos = flat_patch_pos % H # N
    y_pos = flat_patch_pos // H # N
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 4)[: (dim // 4)].float() / dim)) # Hc/4
    x_freqs = torch.outer(x_pos, freqs).float() # N Hc/4
    y_freqs = torch.outer(y_pos, freqs).float() # N Hc/4
    x_cis = torch.polar(torch.ones_like(x_freqs), x_freqs)
    y_cis = torch.polar(torch.ones_like(y_freqs), y_freqs)
    freqs_cis = torch.cat([x_cis.unsqueeze(dim=-1), y_cis.unsqueeze(dim=-1)], dim=-1) # N,Hc/4,2
    freqs_cis = freqs_cis.reshape(end if not use_cls else end + 1, -1)
    # we need to think how to implement this for multi heads.
    # freqs_cis = torch.cat([x_cis, y_cis], dim=-1) # N, Hc/2
    return freqs_cis


def reshape_for_broadcast(freqs_cis, x):
    # x: B N H Hc/2
    # freqs_cis:  N, H*Hc/2 or  N Hc/2
    ndim = x.ndim
    assert 0 <= 1 < ndim
    if freqs_cis.shape[-1] == x.shape[-1]:
        shape = [1 if i == 2 or i == 0 else d for i, d in enumerate(x.shape)]  # 1, N, 1, Hc/2
    else:
        shape = [d if i != 0 else 1 for i, d in enumerate(x.shape)] # 1, N, H, Hc/2
        # B, N, Hc/2
    return freqs_cis.view(*shape)

def apply_rotary_emb(xq, xk, freqs_cis):
    # xq : B N Head Ch_per_Head
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2)) # B N H Hc/2
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3) # B, N, H, Hc
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

from einops import rearrange

class DownSampler(nn.Module):
    def __init__(self, dim, kernel_size, down_factor):
        super().__init__()
        self.down_factor = down_factor
        self.layer = nn.Conv2d(dim, dim, kernel_size, 1, kernel_size//2, groups=dim)
    def forward(self, x):
        x = self.layer(x) + x # down_shortcut
        return rearrange(x, 'b d (h dh) (w dw) -> b (dh dw) (h w) d', dh=self.down_factor, dw=self.down_factor)

class DownSample_Attn(nn.Module):
    # def __init__(self, dim, input_size, down_factor, num_heads, downsampler='dwconv5', down_shortcut=1):
    def __init__(self, dim, down_factor, num_heads, downsampler='dwconv5', down_shortcut=1):
        super().__init__()
        # self.input_resolution = (input_size, input_size)
        self.dim = dim
        self.heads = num_heads
        head_dim = dim // num_heads
        self.dh, self.dw = down_factor, down_factor
        self.to_qkv = nn.Linear(dim, dim*3, bias=False)
        self.to_out = nn.Conv2d(dim, dim, 1)

        # self.scale = head_dim**-.5 # v1
        # v2
        # self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)
        self.logit_scale = nn.Parameter(10 * torch.ones((num_heads, 1, 1)))

        self.downsampler = DownSampler(dim, kernel_size, down_factor)

    def forward(self, x):
        b, _, h, w = x.size()
        x = self.downsampler(x) # [b, r^2, h/r *w/r, d]
        qkv = self.to_qkv(x).chunk(3, dim=-1)

        N = h * w // self.dh // self.dw
        self.freqs_cis = precompute_freqs_cis_2d(self.dim // self.heads, N).to(x.device)
        # q, k input shape: B N H Hc
        q, k = map(lambda t: rearrange(t, 'b p n (h d) -> (b p) n h d', h=self.heads), qkv[:-1])
        v = rearrange(qkv[-1], 'b p n (h d) -> b p h n d', h=self.heads)
        q, k = apply_rotary_emb(q, k, freqs_cis=self.freqs_cis)
        # reshape back
        q = rearrange(q, '(b p) n h d -> b p h n d', b=b)
        k = rearrange(k, '(b p) n h d -> b p h n d', b=b)

        # # v1 attention
        # attn = (q @ k.transpose(-2, -1))
        # attn = attn * self.scale
        # v2 attention
        attn = (F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(-2, -1))
        # logit_scale = torch.clamp(self.logit_scale, max=4.6052).exp()
        # attn = attn * logit_scale
        attn = attn * torch.clamp(self.logit_scale, max=100)

        attn = attn.softmax(dim=-1)
        x = (attn @ v)
        # upsample
        x = rearrange(x, 'b (dh dw) he (h w) d -> b (he d) (h dh) (w dw)', h=h // self.dh, w=w // self.dw, dh=self.dh, dw=self.dw)
        x = self.to_out(x)
        return x


class FeedForward(nn.Module):
    def __init__(self, dim, ffn_expansion_factor, rep=False):
        super().__init__()
        self.rep = rep
        self.bias = bias
        self.hidden_features = hidden_features = int(dim*ffn_expansion_factor)

        self.project_in = nn.Sequential(nn.Conv2d(dim, hidden_features//2, kernel_size=1, bias=bias),
            nn.Conv2d(hidden_features//2, hidden_features, kernel_size=1, bias=bias))
        self.dwconv = nn.ModuleList([
            nn.Conv2d(hidden_features, hidden_features, kernel_size=5, stride=1, padding=2, dilation=1, groups=hidden_features, bias=bias),
            nn.Conv2d(hidden_features, hidden_features, kernel_size=3, stride=1, padding=1, dilation=1, groups=hidden_features, bias=bias),
            nn.Conv2d(hidden_features, hidden_features, kernel_size=1, stride=1, padding=0, dilation=1, groups=hidden_features, bias=bias)
        ])
        self.project_out = nn.Sequential(nn.Conv2d(hidden_features, hidden_features//2, kernel_size=1, bias=bias),
            nn.Conv2d(hidden_features//2, dim, kernel_size=1, bias=bias))

    def forward(self, x):
        x = self.project_in(x)
        x = F.gelu(x)
        out = x
        for module in self.dwconv:
            out = out + module(x)
        x = self.project_out(out)
        return x

#################################################################################
#               Embedding Layers for Timesteps and Class Labels                 #
#################################################################################
import math

class TimestepEmbedder(nn.Module):
    def __init__(self, hidden_size, frequency_embedding_size=256):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size), nn.SiLU(),
            nn.Linear(hidden_size, hidden_size),
        )
        self.frequency_embedding_size = frequency_embedding_size
        nn.init.normal_(self.mlp.weight, std=0.02)

    @staticmethod
    def timestep_embedding(t, dim, max_period=10000):
        """
        Create sinusoidal timestep embeddings.
        :param t: a 1-D Tensor of N indices, one per batch element. These may be fractional.
        :param dim: the dimension of the output.
        :param max_period: controls the minimum frequency of the embeddings.
        :return: an (N, D) Tensor of positional embeddings.
        """
        # https://github.com/openai/glide-text2im/blob/main/glide_text2im/nn.py
        half = dim // 2
        freqs = torch.exp(-math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half).to(device=t.device)
        args = t[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
        return embedding

    def forward(self, t):
        t_freq = self.timestep_embedding(t, self.frequency_embedding_size)
        t_emb = self.mlp(t_freq)
        return t_emb


# class LabelEmbedder(nn.Module):
#     """Embeds class labels into vector representations. Also handles label dropout for classifier-free guidance."""
#     def __init__(self, num_classes, hidden_size, dropout_prob):
#         super().__init__()
#         use_cfg_embedding = dropout_prob > 0
#         self.embedding_table = nn.Embedding(num_classes + use_cfg_embedding, hidden_size)
#         self.num_classes = num_classes
#         self.dropout_prob = dropout_prob
#         nn.init.normal_(self.embedding_table.weight, std=0.02)

#     def token_drop(self, labels, force_drop_ids=None):
#         """
#         Drops labels to enable classifier-free guidance.
#         """
#         if force_drop_ids is None:
#             drop_ids = torch.rand(labels.shape[0], device=labels.device) < self.dropout_prob
#         else:
#             drop_ids = force_drop_ids == 1
#         labels = torch.where(drop_ids, self.num_classes, labels)
#         return labels

#     def forward(self, labels, train, force_drop_ids=None):
#         use_dropout = self.dropout_prob > 0
#         if (train and use_dropout) or (force_drop_ids is not None):
#             labels = self.token_drop(labels, force_drop_ids)
#         embeddings = self.embedding_table(labels)
#         return embeddings


#################################################################################
#                                 Core IPT Model                                #
#################################################################################

class U_DiTBlock(nn.Module):
    """A IPT block with adaptive layer norm zero (adaLN-Zero) conIPTioning."""
    # def __init__(self, input_size, hidden_size, num_heads, mlp_ratio=4.0, down_factor=2, rep=1, ffn_type='rep'):
    def __init__(self, hidden_size, num_heads, mlp_ratio=4.0, down_factor=2, rep=1, ffn_type='rep'):
        super().__init__()
        self.norm1 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        # self.attn = DownSample_Attn(hidden_size, input_size, down_factor=down_factor, num_heads=num_heads)
        self.attn = DownSample_Attn(hidden_size, down_factor=down_factor, num_heads=num_heads)
        self.norm2 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        self.mlp = FeedForward(hidden_size, mlp_ratio, rep=rep)
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(), zero_module(nn.Linear(hidden_size, 6 * hidden_size))
        )

    def forward(self, x, c):
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation(c).chunk(6, dim=1)
        x = x + gate_msa[...,None,None] * self.attn(modulate(self.norm1(x), shift_msa, scale_msa))
        x = x + gate_mlp[...,None,None] * self.mlp(modulate(self.norm2(x), shift_mlp, scale_mlp))
        return x

def modulate(x, shift, scale):
    return x * (1 + scale[...,None,None]) + shift[...,None,None]


# class scale_shift(nn.Module): # FiLM
#     def __init__(self, x_dim, t_dim):
#         super().__init__()
#         self.time_mlp = nn.Sequential(nn.SiLU(), nn.Linear(t_dim, x_dim*2),)

#     def forward(self, x, emb): # [b,c,h,w], [b,emb_dim]
#         scale, shift = self.time_mlp(emb)[..., None, None].chunk(2, dim=1) # [b,t_dim]->[b,2*x_dim,1,1]->[b,x_dim,1,1]
#         return x * (scale + 1) + shift

class FinalLayer(nn.Module):
    def __init__(self, hidden_size, out_channels):
        super().__init__()
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(), zero_module(nn.Linear(hidden_size, 2 * hidden_size))
        )
        self.norm_final = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        self.out_proj = zero_module(nn.Conv2d(hidden_size, out_channels, kernel_size=3, stride=1, padding=1))

    def forward(self, x, c):
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=1)
        x = modulate(self.norm_final(x), shift, scale)
        x = self.out_proj(x)
        return x



class UpDownsample(nn.Module):
    def __init__(self, n_feat, r=1):
        super().__init__()
        if r>1: sample = nn.PixelUnshuffle(r)
        elif r<1: sample = nn.PixelUnshuffle(int(1/r))
        else: sample = nn.Identity()
        self.body = nn.Sequential(nn.Conv2d(n_feat, int(n_feat*r), 3, 1, 1, bias=False), sample)
    def forward(self, x): return self.body(x)

class levelBlock(nn.Module):
    def __init__(self, hidden_size, num_heads, mlp_ratio, down_factor, depth=1, updown=False):
        super().__init__()

        if updown=='down': level = [
            UpDownsample(hidden_size, r=1/2),
            *[U_DiTBlock(hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor, rep=rep, ffn_type=ffn_type) for _ in range(depth)]
        ]
        elif updown=='up': level = [
            nn.Conv2d(hidden_size, hidden_size//2, kernel_size=1),
            *[U_DiTBlock(hidden_size//2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor, rep=rep, ffn_type=ffn_type) for _ in range(depth)]
            UpDownsample(hidden_size//2, r=2)
            ]
        else: level = [
            *[U_DiTBlock(hidden_size, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor, rep=rep, ffn_type=ffn_type) for _ in range(depth)]
        ]
        self.seq = Seq(*level)

    def forward(self, x, cond=None):
        return self.seq(x, cond)


class U_DiT(nn.Module):
    """Diffusion UNet model with a Transformer backbone."""
    def __init__(self, input_size=32, down_factor=2, in_channels=4, hidden_size=1152, depth=[2,5,8,5,2],
        num_heads=16, mlp_ratio=4, class_dropout_prob=0.1, num_classes=1000,
        learn_sigma=True, rep=1, ffn_type='rep', **kwargs):
        super().__init__()
        self.learn_sigma = learn_sigma
        self.in_channels = in_channels
        self.out_channels = in_channels * 2 if learn_sigma else in_channels
        self.num_heads = num_heads

        down_factor = down_factor if isinstance(down_factor, list) else [down_factor] * 5

        self.x_embedder = nn.Conv2d(in_channels, hidden_size, kernel_size=3, stride=1, padding=1)
        # Initialize patch_embed like nn.Linear (instead of nn.Conv2d):
        w = self.x_embedder.weight.data
        nn.init.xavier_uniform_(w.view([w.shape[0], -1]))
        nn.init.constant_(self.x_embedder.bias, 0)


        self.t_embedder_1 = TimestepEmbedder(hidden_size)
        # self.y_embedder_1 = LabelEmbedder(num_classes, hidden_size, class_dropout_prob)
        self.y_embedder_1 = nn.Embedding(num_classes, hidden_size)

        self.t_embedder_2 = TimestepEmbedder(hidden_size*2)
        self.y_embedder_2 = LabelEmbedder(num_classes, hidden_size*2, class_dropout_prob)

        self.t_embedder_3 = TimestepEmbedder(hidden_size*4)
        self.y_embedder_3 = LabelEmbedder(num_classes, hidden_size*4, class_dropout_prob)

        # encoder-1
        self.encoder_level_1 = nn.ModuleList([
            U_DiTBlock(input_size, hidden_size, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[0], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[0])
        ])


        # encoder-2
        self.down1_2 = Downsample(hidden_size)
        self.encoder_level_2 = nn.ModuleList([
            U_DiTBlock(input_size//2, hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[1], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[1])
        ])

        # latent
        self.down2_3 = Downsample(hidden_size*2)
        self.latent = nn.ModuleList([
            U_DiTBlock(input_size//4, hidden_size*4, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[2], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[2])
        ])
        self.up3_2 = Upsample(int(hidden_size*4))  ## From Level 4 to Level 3

        # decoder-2
        self.reduce_chan_level2 = nn.Conv2d(int(hidden_size*4), int(hidden_size*2), kernel_size=1)
        self.decoder_level_2 = nn.ModuleList([
            U_DiTBlock(input_size//2, hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[3], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[3])
        ])
        self.up2_1 = Upsample(int(hidden_size*2))  ## From Level 4 to Level 3

        # decoder-1
        self.reduce_chan_level1 = nn.Conv2d(int(hidden_size*2), int(hidden_size*2), kernel_size=1)
        self.decoder_level_1 = nn.ModuleList([
            U_DiTBlock(input_size, hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[4], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[4])
        ])

        self.output = nn.Conv2d(int(hidden_size*2), int(hidden_size*2), kernel_size=3, stride=1, padding=1)
        self.final_layer = FinalLayer(hidden_size*2, self.out_channels)
        self.apply(self._basic_init)

    def _basic_init(self, module):
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)


    def forward(self, x, t, y): # [b,c,h,w], time [b], class label [b]
        x = self.x_embedder(x) # (N, C, H, W)

        t1, t2, t3 = self.t_embedder_1(t), self.t_embedder_2(t), self.t_embedder_3(t) # (N, C, 1, 1)
        y1, y2, y3 = self.y_embedder_1(y, self.training), self.y_embedder_2(y, self.training), self.y_embedder_3(y, self.training) # (N, C, 1, 1)
        c1, c2, c3 = t1 + y1, t2 + y2, t3 + y3 # (N, D, 1, 1)

        blocks = []

        for block in self.encoder_level_1:
            x = block(x, c1)
        blocks.append(x)

        x = self.down1_2(x)
        for block in self.encoder_level_2:
            x = block(x, c2)
        blocks.append(x)

        x = self.down2_3(x)
        for block in self.latent:
            x = block(x, c3)
        x = self.up3_2(x)

        x = torch.cat([x, blocks.pop()], 1)
        x = self.reduce_chan_level2(x)
        for block in self.decoder_level_2:
            x = block(x, c2)
        x = self.up2_1(x)

        x = torch.cat([x, blocks.pop()], 1)
        x = self.reduce_chan_level1(x)
        for block in self.decoder_level_1:
            x = block(x, c2)

        # output
        x = self.output(x)
        x = self.final_layer(x, c2)                # (N, T, patch_size ** 2 * out_channels)
        return x



def U_DiT_S(**kwargs):
    return U_DiT(down_factor=2, hidden_size=96, num_heads=4, depth=[2,5,8,5,2], ffn_type='rep', rep=1, mlp_ratio=2, downsampler='dwconv5', down_shortcut=1)
def U_DiT_B(**kwargs): return U_DiT(down_factor=2, hidden_size=192, num_heads=8,
def U_DiT_L(**kwargs): return U_DiT(down_factor=2, hidden_size=384, num_heads=16,


# Ablations on downsampler
model = U_DiT_S()
model.load_state_dict(torch.load('pt/newpt/U-DiT/U-DiT-S-400k.pt'))

model.cuda()
model.eval()

inputs = torch.rand(1, 4, 32, 32).cuda()
t = torch.ones(1).int().cuda()
y = torch.ones(1).int().cuda()

model(inputs, t, y)
out = model(inputs, t, y)

flops = profile_macs(model, (inputs, t, y))
print(f'FLOPS: {flops/1e6:.2f}')


model.train()
print(f"output : {out.size()}")

# backward test
out = model(inputs, t, y)
gt = torch.rand(1, 8, 32, 32).cuda()

loss = torch.mean(out-gt)
loss.backward()


In [ ]:
# @title YuchuanTian/U-DiT save
# https://github.com/YuchuanTian/U-DiT/blob/main/udit_models.py

# Copyright 2024 Huawei Technologies Co., Ltd
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================
# --------------------------------------------------------
# References:
# GLIDE: https://github.com/openai/glide-text2im
# MAE: https://github.com/facebookresearch/mae/blob/main/models_mae.py
# --------------------------------------------------------

import torch
import torch.nn as nn
import numpy as np
import math
import torch.nn.functional as F
# from timm.models.vision_transformer import PatchEmbed
# from timm.models.layers import LayerNorm2d

from pdb import set_trace as stx
import numbers
# from torchprofile import profile_macs
from einops import rearrange
import einops

class LayerNorm2d(nn.LayerNorm):
    def __init__(self, num_channels, eps=1e-6, affine=True):
        super().__init__(num_channels, eps=eps, elementwise_affine=affine)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.permute(0, 2, 3, 1)
        x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        x = x.permute(0, 3, 1, 2)
        return x

def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # From: https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/layers/weight_init.py
    # Cut & paste from Pytorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn(
            'mean is more than 2 std from [a, b] in nn.init.trunc_normal_. '
            'The distribution of values may be incorrect.',
            stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        low = norm_cdf((a - mean) / std)
        up = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [low, up], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * low - 1, 2 * up - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution.

    From: https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/layers/weight_init.py

    The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value

    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)

#######################################3###################################
def precompute_freqs_cis_2d(dim: int, end: int, theta: float = 10000.0, scale=1.0, use_cls=False):
    H = int( end**0.5 )
    # assert  H * H == end
    flat_patch_pos = torch.arange(0 if not use_cls else -1, end) # N = end
    x_pos = flat_patch_pos % H # N
    y_pos = flat_patch_pos // H # N
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 4)[: (dim // 4)].float() / dim)) # Hc/4
    x_freqs = torch.outer(x_pos, freqs).float() # N Hc/4
    y_freqs = torch.outer(y_pos, freqs).float() # N Hc/4
    x_cis = torch.polar(torch.ones_like(x_freqs), x_freqs)
    y_cis = torch.polar(torch.ones_like(y_freqs), y_freqs)
    freqs_cis = torch.cat([x_cis.unsqueeze(dim=-1), y_cis.unsqueeze(dim=-1)], dim=-1) # N,Hc/4,2
    freqs_cis = freqs_cis.reshape(end if not use_cls else end + 1, -1)
    # we need to think how to implement this for multi heads.
    # freqs_cis = torch.cat([x_cis, y_cis], dim=-1) # N, Hc/2
    return freqs_cis


def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    # x: B N H Hc/2
    # freqs_cis:  N, H*Hc/2 or  N Hc/2
    ndim = x.ndim
    assert 0 <= 1 < ndim

    if freqs_cis.shape[-1] == x.shape[-1]:
        shape = [1 if i == 2 or i == 0 else d for i, d in enumerate(x.shape)]  # 1, N, 1, Hc/2
    else:
        shape = [d if i != 0 else 1 for i, d in enumerate(x.shape)] # 1, N, H, Hc/2
        # B, N, Hc/2
    return freqs_cis.view(*shape)

def apply_rotary_emb(
        xq: torch.Tensor,
        xk: torch.Tensor,
        freqs_cis: torch.Tensor,
):
    # xq : B N Head Ch_per_Head
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2)) # B N H Hc/2
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3) # B, N, H, Hc
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)


class DownSampler(nn.Module):
    def __init__(self, *args, **kwargs):
        ''' Required kwargs: down_factor, downsampler'''
        super().__init__()
        self.down_factor = kwargs.pop('down_factor')
        self.downsampler = kwargs.pop('downsampler')
        self.down_shortcut = kwargs.pop('down_shortcut')
        self.layer = nn.Conv2d(*args, **kwargs)

    def forward(self, x):
        x = self.layer(x) + (x if self.down_shortcut else 0)
        return rearrange(x, 'b d (h dh) (w dw) -> b (dh dw) (h w) d', dh=self.down_factor, dw=self.down_factor)


class DownSample_Attn(nn.Module):
    def __init__(self, dim, input_size, down_factor, num_heads,
                bias=False, posemb_type=None, attn_type=None, downsampler=None, down_shortcut=False, **kwargs):
        super(DownSample_Attn, self).__init__()
        if kwargs != dict(): # is not empty
            print(f'Kwargs: {kwargs}')

        self.input_resolution = (input_size, input_size)
        self.dim = dim
        self.heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5
        self.dh, self.dw = down_factor, down_factor
        self.to_qkv = nn.Linear(dim, dim * 3, bias=bias)

        self.to_out = nn.Conv2d(dim, dim, 1)

        # v2
        self.attn_type = attn_type
        if attn_type == 'v2':
            self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)

        # posemb
        self.posemb_type = posemb_type
        # if posemb_type is not None:
        #     print(f'Pos Emb {posemb_type} activated...')

        # posemb type
        if self.posemb_type == 'rope2d':
            self.freqs_cis = None

        # downsampler
        downsampler_ker_size = int(downsampler[-1])
        downsampler_padding = (int(downsampler[-1])-1)//2
        self.downsampler = DownSampler(dim, dim, kernel_size=downsampler_ker_size, stride=1, padding=downsampler_padding, groups=dim, down_factor=down_factor, downsampler=downsampler, down_shortcut=down_shortcut)



    def forward(self, x):
        b, _, h, w = x.size()
        x = self.downsampler(x)
        qkv = self.to_qkv(x).chunk(3, dim=-1)

        if self.posemb_type == 'rope2d':
            N = h * w // self.dh // self.dw
            if self.freqs_cis is None or self.freqs_cis.shape[0] != N:
                self.freqs_cis = precompute_freqs_cis_2d(self.dim // self.heads, N).to(x.device)
            # q, k input shape: B N H Hc
            q, k = map(lambda t: rearrange(t, 'b p n (h d) -> (b p) n h d', h=self.heads), qkv[:-1])
            v = rearrange(qkv[2], 'b p n (h d) -> b p h n d', h=self.heads)
            q, k = apply_rotary_emb(q, k, freqs_cis=self.freqs_cis)
            # reshape back
            q = rearrange(q, '(b p) n h d -> b p h n d', b=b)
            k = rearrange(k, '(b p) n h d -> b p h n d', b=b)
        else:
            q, k, v = map(lambda t: rearrange(t, 'b p n (h d) -> b p h n d', h=self.heads), qkv)

        if self.attn_type is None: # v1 attention
            attn = (q @ k.transpose(-2, -1))
            attn = attn * self.scale
        elif self.attn_type == 'v2': # v2 attention
            attn = (F.normalize(q, dim=-1) @ F.normalize(k, dim=-1).transpose(-2, -1))
            logit_scale = torch.clamp(self.logit_scale, max=4.6052).exp()
            attn = attn * logit_scale

        attn = attn.softmax(dim=-1)
        x = (attn @ v)
        # upsample
        x = rearrange(x, 'b (dh dw) he (h w) d -> b (he d) (h dh) (w dw)', h=h // self.dh, w=w // self.dw, dh=self.dh, dw=self.dw)

        x = self.to_out(x)
        return x


class FeedForward(nn.Module):
    def __init__(self, dim, ffn_expansion_factor, bias=True, rep=False):
        super(FeedForward, self).__init__()

        self.rep = rep
        self.bias = bias

        self.hidden_features = hidden_features = int(dim*ffn_expansion_factor)

        # buffer
        self.buffer = False # False: empty
        self.project_in_weight = None
        self.project_in_bias = None
        self.dwconv_weight = None
        self.dwconv_bias = None
        self.project_out_weight = None
        self.project_out_bias = None

        if rep:
            self.project_in = nn.Sequential(nn.Conv2d(dim, hidden_features//2, kernel_size=1, bias=bias),
                                            nn.Conv2d(hidden_features//2, hidden_features, kernel_size=1, bias=bias))

            self.dwconv = nn.ModuleList([
                nn.Conv2d(hidden_features, hidden_features, kernel_size=5, stride=1, padding=2, dilation=1, groups=hidden_features, bias=bias),
                nn.Conv2d(hidden_features, hidden_features, kernel_size=3, stride=1, padding=1, dilation=1, groups=hidden_features, bias=bias),
                nn.Conv2d(hidden_features, hidden_features, kernel_size=1, stride=1, padding=0, dilation=1, groups=hidden_features, bias=bias)
            ])

            self.project_out = nn.Sequential(nn.Conv2d(hidden_features, hidden_features//2, kernel_size=1, bias=bias),
                                             nn.Conv2d(hidden_features//2, dim, kernel_size=1, bias=bias))

        else:
            self.project_in = nn.Conv2d(dim, hidden_features, kernel_size=1, bias=bias)
            self.dwconv = nn.Conv2d(hidden_features, hidden_features, kernel_size=5, stride=1, padding=2, groups=hidden_features, bias=bias)
            self.project_out = nn.Conv2d(hidden_features, dim, kernel_size=1, bias=bias)

    def clear_buffer(self):
        self.buffer = False # False: empty
        self.project_in_weight = None
        self.project_in_bias = None
        self.dwconv_weight = None
        self.dwconv_bias = None
        self.project_out_weight = None
        self.project_out_bias = None

    def add_buffer(self):
        self.buffer = True

        w1 = self.project_in[0].weight.squeeze().detach()
        w2 = self.project_in[1].weight.squeeze().detach()
        self.project_in_weight = (w2 @ w1).unsqueeze(-1).unsqueeze(-1)
        self.project_in_bias = None

        if self.bias:
            b1 = self.project_in[0].bias.detach()
            b2 = self.project_in[1].bias.detach()
            self.project_in_bias = (w2 @ b1 + b2)

        self.dwconv_weight = self.dwconv[0].weight.detach()
        self.dwconv_weight[:, :, 1:4, 1:4] += self.dwconv[1].weight.detach()
        self.dwconv_weight[:, :, 2:3, 2:3] += (self.dwconv[2].weight.detach() + 1.) # skip connection

        self.dwconv_bias = None
        if self.bias:
            self.dwconv_bias = self.dwconv[0].bias.detach() + self.dwconv[1].bias.detach() + self.dwconv[2].bias.detach()

        w1 = self.project_out[0].weight.squeeze().detach()
        w2 = self.project_out[1].weight.squeeze().detach()
        self.project_out_weight = (w2 @ w1).unsqueeze(-1).unsqueeze(-1)
        self.project_out_bias = None

        if self.bias:
            b1 = self.project_out[0].bias.detach()
            b2 = self.project_out[1].bias.detach()
            self.project_out_bias = (w2 @ b1 + b2)


    def train(self, mode: bool = True):
        if not isinstance(mode, bool):
            raise ValueError("training mode is expected to be boolean")
        self.training = mode
        if mode:
            self.clear_buffer() # added: clear rep buffer
        else:
            self.add_buffer() # added: add rep buffer
        for module in self.children():
            module.train(mode)
        return self


    def forward(self, x):
        if (not self.rep) or (self.rep and self.training): # debugged: 2024/6/4
            x = self.project_in(x)
            x = F.gelu(x)
            if self.rep:
                out = x
                for module in self.dwconv:
                    out = out + module(x)
            else:
                out = self.dwconv(x)
            x = self.project_out(out)
        else: # eval & rep
            x = F.conv2d(x, self.project_in_weight, self.project_in_bias) # project_in
            x = F.gelu(x) # debugged: 2024/6/4
            x = F.conv2d(x, self.dwconv_weight, self.dwconv_bias, padding=2, groups=self.hidden_features)
            x = F.conv2d(x, self.project_out_weight, self.project_out_bias) # project_out

        return x




class Mlp(nn.Module):
    """ MLP as used in Vision Transformer, MLP-Mixer and related networks
    """
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, bias=True, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        bias = (bias, bias)
        drop_probs = (drop, drop)

        self.fc1 = nn.Conv2d(in_features, hidden_features, 1, bias=bias[0])
        self.act = act_layer()
        self.drop1 = nn.Dropout(drop_probs[0])
        self.fc2 = nn.Conv2d(hidden_features, out_features, 1, bias=bias[1])
        self.drop2 = nn.Dropout(drop_probs[1])

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        return x


def modulate(x, shift, scale):
    return x * (1 + scale.unsqueeze(-1).unsqueeze(-1)) + shift.unsqueeze(-1).unsqueeze(-1)

#################################################################################
#               Embedding Layers for Timesteps and Class Labels                 #
#################################################################################

class TimestepEmbedder(nn.Module):
    """
    Embeds scalar timesteps into vector representations.
    """
    def __init__(self, hidden_size, frequency_embedding_size=256):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size,bias=True),
            nn.SiLU(),
            nn.Linear(hidden_size, hidden_size, bias=True),
        )
        self.frequency_embedding_size = frequency_embedding_size

    @staticmethod
    def timestep_embedding(t, dim, max_period=10000):
        """
        Create sinusoidal timestep embeddings.
        :param t: a 1-D Tensor of N indices, one per batch element.
                          These may be fractional.
        :param dim: the dimension of the output.
        :param max_period: controls the minimum frequency of the embeddings.
        :return: an (N, D) Tensor of positional embeddings.
        """
        # https://github.com/openai/glide-text2im/blob/main/glide_text2im/nn.py
        half = dim // 2
        freqs = torch.exp(
            -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
        ).to(device=t.device)
        args = t[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
        return embedding

    def forward(self, t):
        t_freq = self.timestep_embedding(t, self.frequency_embedding_size)
        t_emb = self.mlp(t_freq)
        return t_emb


class LabelEmbedder(nn.Module):
    """
    Embeds class labels into vector representations. Also handles label dropout for classifier-free guidance.
    """
    def __init__(self, num_classes, hidden_size, dropout_prob):
        super().__init__()
        use_cfg_embedding = dropout_prob > 0
        self.embedding_table = nn.Embedding(num_classes + use_cfg_embedding, hidden_size)
        self.num_classes = num_classes
        self.dropout_prob = dropout_prob

    def token_drop(self, labels, force_drop_ids=None):
        """
        Drops labels to enable classifier-free guidance.
        """
        if force_drop_ids is None:
            drop_ids = torch.rand(labels.shape[0], device=labels.device) < self.dropout_prob
        else:
            drop_ids = force_drop_ids == 1
        labels = torch.where(drop_ids, self.num_classes, labels)
        return labels

    def forward(self, labels, train, force_drop_ids=None):
        use_dropout = self.dropout_prob > 0
        if (train and use_dropout) or (force_drop_ids is not None):
            labels = self.token_drop(labels, force_drop_ids)
        embeddings = self.embedding_table(labels)
        return embeddings


#################################################################################
#                                 Core IPT Model                                #
#################################################################################

class U_DiTBlock(nn.Module):
    """
    A IPT block with adaptive layer norm zero (adaLN-Zero) conIPTioning.
    """
    def __init__(self, input_size, hidden_size, num_heads, mlp_ratio=4.0, down_factor=2, rep=1, ffn_type='rep', **kwargs):
        '''
        Args:
        dw_expand: For NAFNet
        dw_type: NAFNet dwconv type
        posemb_type: pos emb type of IPT related
        '''
        super().__init__()


        self.norm11 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)

        self.attn1 = DownSample_Attn(hidden_size, input_size, down_factor=down_factor, num_heads=num_heads, bias=True, **kwargs)

        self.norm21 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        if ffn_type == 'rep':
            self.mlp1 = FeedForward(hidden_size, mlp_ratio, rep=rep)
        elif ffn_type == 'basic':
            approx_gelu = lambda: nn.GELU(approximate="tanh")
            mlp_hidden_dim = int(hidden_size * mlp_ratio)
            self.mlp1 = Mlp(in_features=hidden_size, hidden_features=mlp_hidden_dim, act_layer=approx_gelu, drop=0)
        else:
            raise NotImplementedError(f'FFN type not implemented: {ffn_type}!')
        self.adaLN_modulation1 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 6 * hidden_size, bias=True)
        )


        self.norm12 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)

        self.attn2 = DownSample_Attn(hidden_size, input_size, down_factor=down_factor, num_heads=num_heads, bias=True, **kwargs)

        self.norm22 = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        if ffn_type == 'rep':
            self.mlp2 = FeedForward(hidden_size, mlp_ratio, rep=rep)
        elif ffn_type == 'basic':
            approx_gelu = lambda: nn.GELU(approximate="tanh")
            mlp_hidden_dim = int(hidden_size * mlp_ratio)
            self.mlp2 = Mlp(in_features=hidden_size, hidden_features=mlp_hidden_dim, act_layer=approx_gelu, drop=0)
        else:
            raise NotImplementedError(f'FFN type not implemented: {ffn_type}!')
        self.adaLN_modulation2 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 6 * hidden_size, bias=True)
        )

    def forward(self, x, c):

        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation1(c).chunk(6, dim=1)
        x = x + gate_msa.unsqueeze(-1).unsqueeze(-1) * self.attn1(modulate(self.norm11(x), shift_msa, scale_msa))
        x = x + gate_mlp.unsqueeze(-1).unsqueeze(-1) * self.mlp1(modulate(self.norm21(x), shift_mlp, scale_mlp))

        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation2(c).chunk(6, dim=1)
        x = x + gate_msa.unsqueeze(-1).unsqueeze(-1) * self.attn2(modulate(self.norm12(x), shift_msa, scale_msa))
        x = x + gate_mlp.unsqueeze(-1).unsqueeze(-1) * self.mlp2(modulate(self.norm22(x), shift_mlp, scale_mlp))
        return x

class FinalLayer(nn.Module):
    """
    The final layer of IPT.
    """
    def __init__(self, hidden_size, out_channels):
        super().__init__()
        self.norm_final = LayerNorm2d(hidden_size, affine=False, eps=1e-6)
        self.out_proj = nn.Conv2d(hidden_size, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 2 * hidden_size, bias=True)
        )

    def forward(self, x, c):
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=1)
        x = modulate(self.norm_final(x), shift, scale)
        x = self.out_proj(x)
        return x

class OverlapPatchEmbed(nn.Module):
    def __init__(self, in_c=3, embed_dim=48, bias=False):
        super(OverlapPatchEmbed, self).__init__()

        self.proj = nn.Conv2d(in_c, embed_dim, kernel_size=3, stride=1, padding=1, bias=bias)

    def forward(self, x):
        x = self.proj(x)

        return x

class Downsample(nn.Module):
    def __init__(self, n_feat):
        super(Downsample, self).__init__()

        self.body = nn.Sequential(nn.Conv2d(n_feat, n_feat // 2, kernel_size=3, stride=1, padding=1, bias=False),
                                  nn.PixelUnshuffle(2))

    def forward(self, x):
        return self.body(x)

class Upsample(nn.Module):
    def __init__(self, n_feat):
        super(Upsample, self).__init__()

        self.body = nn.Sequential(nn.Conv2d(n_feat, n_feat * 2, kernel_size=3, stride=1, padding=1, bias=False),
                                  nn.PixelShuffle(2))

    def forward(self, x):
        return self.body(x)

class U_DiT(nn.Module):
    """
    Diffusion UNet model with a Transformer backbone.
    """
    def __init__(
        self,
        input_size=32,
        down_factor=2,
        in_channels=4,
        hidden_size=1152,
        depth=[2,5,8,5,2],
        num_heads=16,
        mlp_ratio=4,
        class_dropout_prob=0.1,
        num_classes=1000,
        learn_sigma=True,
        rep=1,
        ffn_type='rep',
        **kwargs
    ):
        super().__init__()
        self.learn_sigma = learn_sigma
        self.in_channels = in_channels
        self.out_channels = in_channels * 2 if learn_sigma else in_channels
        self.num_heads = num_heads

        down_factor = down_factor if isinstance(down_factor, list) else [down_factor] * 5


        self.x_embedder = OverlapPatchEmbed(in_channels, hidden_size, bias=True)
        self.t_embedder_1 = TimestepEmbedder(hidden_size)
        self.y_embedder_1 = LabelEmbedder(num_classes, hidden_size, class_dropout_prob)

        self.t_embedder_2 = TimestepEmbedder(hidden_size*2)
        self.y_embedder_2 = LabelEmbedder(num_classes, hidden_size*2, class_dropout_prob)

        self.t_embedder_3 = TimestepEmbedder(hidden_size*4)
        self.y_embedder_3 = LabelEmbedder(num_classes, hidden_size*4, class_dropout_prob)

        # encoder-1
        self.encoder_level_1 = nn.ModuleList([
            U_DiTBlock(input_size, hidden_size, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[0], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[0])
        ])
        self.down1_2 = Downsample(hidden_size)

        # encoder-2
        self.encoder_level_2 = nn.ModuleList([
            U_DiTBlock(input_size//2, hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[1], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[1])
        ])
        self.down2_3 = Downsample(hidden_size*2)

        # latent
        self.latent = nn.ModuleList([
            U_DiTBlock(input_size//4, hidden_size*4, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[2], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[2])
        ])

        # decoder-2
        self.up3_2 = Upsample(int(hidden_size*4))  ## From Level 4 to Level 3
        self.reduce_chan_level2 = nn.Conv2d(int(hidden_size*4), int(hidden_size*2), kernel_size=1, bias=True)
        self.decoder_level_2 = nn.ModuleList([
            U_DiTBlock(input_size//2, hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[3], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[3])
        ])

        # decoder-1
        self.up2_1 = Upsample(int(hidden_size*2))  ## From Level 4 to Level 3
        self.reduce_chan_level1 = nn.Conv2d(int(hidden_size*2), int(hidden_size*2), kernel_size=1, bias=True)
        self.decoder_level_1 = nn.ModuleList([
            U_DiTBlock(input_size, hidden_size*2, num_heads, mlp_ratio=mlp_ratio, down_factor=down_factor[4], rep=rep, ffn_type=ffn_type, **kwargs) for _ in range(depth[4])
        ])

        self.output = nn.Conv2d(int(hidden_size*2), int(hidden_size*2), kernel_size=3, stride=1, padding=1, bias=True)

        self.final_layer = FinalLayer(hidden_size*2, self.out_channels)
        self.initialize_weights()

    def initialize_weights(self):
        # Initialize transformer layers:
        def _basic_init(module):
            if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
                torch.nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
        self.apply(_basic_init)

        # Initialize patch_embed like nn.Linear (instead of nn.Conv2d):
        w = self.x_embedder.proj.weight.data
        nn.init.xavier_uniform_(w.view([w.shape[0], -1]))
        nn.init.constant_(self.x_embedder.proj.bias, 0)

        # Initialize label embedding table:
        nn.init.normal_(self.y_embedder_1.embedding_table.weight, std=0.02)
        nn.init.normal_(self.y_embedder_2.embedding_table.weight, std=0.02)
        nn.init.normal_(self.y_embedder_3.embedding_table.weight, std=0.02)

        # Initialize timestep embedding MLP:
        nn.init.normal_(self.t_embedder_1.mlp[0].weight, std=0.02)
        nn.init.normal_(self.t_embedder_1.mlp[2].weight, std=0.02)

        nn.init.normal_(self.t_embedder_2.mlp[0].weight, std=0.02)
        nn.init.normal_(self.t_embedder_2.mlp[2].weight, std=0.02)

        nn.init.normal_(self.t_embedder_3.mlp[0].weight, std=0.02)
        nn.init.normal_(self.t_embedder_3.mlp[2].weight, std=0.02)

        # Zero-out adaLN modulation layers in IPT blocks:
        blocks = self.encoder_level_1 + self.encoder_level_2 + self.latent + self.decoder_level_2 + self.decoder_level_1
        for block in blocks:
            nn.init.constant_(block.adaLN_modulation1[-1].weight, 0)
            nn.init.constant_(block.adaLN_modulation1[-1].bias, 0)
            nn.init.constant_(block.adaLN_modulation2[-1].weight, 0)
            nn.init.constant_(block.adaLN_modulation2[-1].bias, 0)

        # Zero-out output layers:
        nn.init.constant_(self.final_layer.adaLN_modulation[-1].weight, 0)
        nn.init.constant_(self.final_layer.adaLN_modulation[-1].bias, 0)
        nn.init.constant_(self.final_layer.out_proj.weight, 0)
        nn.init.constant_(self.final_layer.out_proj.bias, 0)

    def forward(self, x, t, y):
        """
        Forward pass of U-DiT.
        x: (N, C, H, W) tensor of spatial inputs (images or latent representations of images)
        t: (N,) tensor of diffusion timesteps
        y: (N,) tensor of class labels
        """
        x = self.x_embedder(x)                   # (N, C, H, W)
        t1 = self.t_embedder_1(t)    # (N, C, 1, 1)
        y1 = self.y_embedder_1(y, self.training)    # (N, C, 1, 1)
        c1 = t1 + y1                                # (N, D, 1, 1)

        t2 = self.t_embedder_2(t)    # (N, C, 1, 1)
        y2 = self.y_embedder_2(y, self.training)    # (N, C, 1, 1)
        c2 = t2 + y2                                # (N, D, 1, 1)

        t3 = self.t_embedder_3(t)    # (N, C, 1, 1)
        y3 = self.y_embedder_3(y, self.training)    # (N, C, 1, 1)
        c3 = t3 + y3                                # (N, D, 1, 1)

        # encoder_1
        out_enc_level1 = x
        for block in self.encoder_level_1:
            out_enc_level1 = block(out_enc_level1, c1)
        inp_enc_level2 = self.down1_2(out_enc_level1)

        # encoder_2
        out_enc_level2 = inp_enc_level2
        for block in self.encoder_level_2:
            out_enc_level2 = block(out_enc_level2, c2)
        inp_enc_level3 = self.down2_3(out_enc_level2)

        # latent
        latent = inp_enc_level3
        for block in self.latent:
            latent = block(latent, c3)

        # decoder_2
        inp_dec_level2 = self.up3_2(latent)
        inp_dec_level2 = torch.cat([inp_dec_level2, out_enc_level2], 1)
        inp_dec_level2 = self.reduce_chan_level2(inp_dec_level2)
        out_dec_level2 = inp_dec_level2
        for block in self.decoder_level_2:
            out_dec_level2 = block(out_dec_level2, c2)

        # decoder_1
        inp_dec_level1 = self.up2_1(out_dec_level2)
        inp_dec_level1 = torch.cat([inp_dec_level1, out_enc_level1], 1)
        inp_dec_level1 = self.reduce_chan_level1(inp_dec_level1)
        out_dec_level1 = inp_dec_level1
        for block in self.decoder_level_1:
            out_dec_level1 = block(out_dec_level1, c2)

        # output
        x = self.output(out_dec_level1)

        x = self.final_layer(x, c2)                # (N, T, patch_size ** 2 * out_channels)

        return x

    def forward_with_cfg(self, x, t, y, cfg_scale):
        """
        Forward pass of IPT, but also batches the unconIPTional forward pass for classifier-free guidance.
        """
        # https://github.com/openai/glide-text2im/blob/main/notebooks/text2im.ipynb
        half = x[: len(x) // 2]
        combined = torch.cat([half, half], dim=0)
        model_out = self.forward(combined, t, y)
        # For exact reproducibility reasons, we apply classifier-free guidance on only
        # three channels by default. The standard approach to cfg applies it to all channels.
        # This can be done by uncommenting the following line and commenting-out the line following that.
        # eps, rest = model_out[:, :self.in_channels], model_out[:, self.in_channels:]
        eps, rest = model_out[:, :3], model_out[:, 3:]
        cond_eps, uncond_eps = torch.split(eps, len(eps) // 2, dim=0)
        half_eps = uncond_eps + cfg_scale * (cond_eps - uncond_eps)
        eps = torch.cat([half_eps, half_eps], dim=0)
        return torch.cat([eps, rest], dim=1)

#################################################################################
#                                   U-DITs Configs                                  #
#################################################################################
def U_DiT_custom(**kwargs):
    return U_DiT(**kwargs)

def U_DiT_S(**kwargs):
    return U_DiT(down_factor=2, hidden_size=96, num_heads=4, depth=[2,5,8,5,2], ffn_type='rep', rep=1, mlp_ratio=2, attn_type='v2', posemb_type='rope2d', downsampler='dwconv5', down_shortcut=1)

def U_DiT_B(**kwargs):
    return U_DiT(down_factor=2, hidden_size=192, num_heads=8, depth=[2,5,8,5,2], ffn_type='rep', rep=1, mlp_ratio=2, attn_type='v2', posemb_type='rope2d', downsampler='dwconv5', down_shortcut=1)

def U_DiT_L(**kwargs):
    return U_DiT(down_factor=2, hidden_size=384, num_heads=16, depth=[2,5,8,5,2], ffn_type='rep', rep=1, mlp_ratio=2, attn_type='v2', posemb_type='rope2d', downsampler='dwconv5', down_shortcut=1)

DiT_models = {
    'U-DiT-custom': U_DiT_custom,
    'U-DiT-S': U_DiT_S, # U-DiT-S
    'U-DiT-B': U_DiT_B, # U-DiT-B
    'U-DiT-L': U_DiT_L, # U-DiT-L
}


if __name__=="__main__":
    from torchprofile import profile_macs
    import warnings

    # Ablations on downsampler
    model = U_DiT_S()
    model.load_state_dict(torch.load('pt/newpt/U-DiT/U-DiT-S-400k.pt'))

    model.cuda()
    model.eval()

    inputs = torch.rand(1, 4, 32, 32).cuda()
    t = torch.ones(1).int().cuda()
    y = torch.ones(1).int().cuda()

    model(inputs, t, y)
    out = model(inputs, t, y)

    flops = profile_macs(model, (inputs, t, y))
    print(f'FLOPS: {flops/1e6:.2f}')


    model.train()
    print(f"output : {out.size()}")

    # backward test
    out = model(inputs, t, y)
    gt = torch.rand(1, 8, 32, 32).cuda()

    loss = torch.mean(out-gt)
    loss.backward()
